# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Importing Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [6]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS events
                        WITH REPLICATION =
                        {'class': 'SimpleStrategy', 'replication_factor': 1 }""")
except Exception as e:
    print(e)

#### Setting Keyspace

In [7]:
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Creating table storing artist, song title and song's length inforation for specific sessions and item number in session, partitioning by session id, clustering by item number in session
    

In [8]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_len decimal, PRIMARY KEY (session_id, item_in_session))"
session.execute(query)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_session (session_id, item_in_session, artist, song_title, song_len) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        #Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Query returning the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [10]:
query = "SELECT artist, song_title, song_len FROM song_info_by_session WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_len=Decimal('495.3073'))


#### Creating table storing artist name, song title, used name information for specific used id, session and item in session,
#### partitioning by composite PK user id and session id, clustering by item number in session

In [11]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user_per_userid_sessionid "
query += "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, user_fname text, user_lname text, PRIMARY KEY ((user_id, session_id), item_in_session))"
session.execute(query)
               

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song_user_per_userid_sessionid (user_id, session_id, item_in_session, artist_name, song_title, user_fname, user_lname) "
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        #Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Query returning the artist, song title and song's length in the music app history that was heard by user id = 10 during sessionId = 182

In [13]:
query = "SELECT artist_name, song_title, user_fname, user_lname FROM artist_song_user_per_userid_sessionid WHERE user_id = 10 AND session_id = 182 ORDER BY item_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_fname='Sylvie', user_lname='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_fname='Sylvie', user_lname='Cruz')


#### Creating table storing user name per song title and session id, partitioning by composite PK song title and session id

In [14]:
query = "CREATE TABLE IF NOT EXISTS usernames_per_song "
query += "(song_title text, session_id int, user_fname text, user_lname text, PRIMARY KEY(song_title, session_id))"
session.execute(query)                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO usernames_per_song (song_title, session_id, user_fname, user_lname) "
        query += "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[8]), line[1], line[4]))

#### Query returning every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
query = "SELECT user_fname, user_lname FROM usernames_per_song WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(user_fname='Sara', user_lname='Johnson')
Row(user_fname='Jacqueline', user_lname='Lynch')
Row(user_fname='Tegan', user_lname='Levine')


### Drop the tables before closing out the sessions

In [17]:
query = "DROP TABLE IF EXISTS song_info_by_session"
session.execute(query)

In [18]:
query = "DROP TABLE IF EXISTS artist_song_user_per_userid_sessionid"
session.execute(query)

In [19]:
query = "DROP TABLE IF EXISTS usernames_per_song"
session.execute(query)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()